
1. View example figures with label in 3 x 3 grids
2. Use data augmentation to increase data size
3. Use callback to monitor training.
4. Apply early stopping, learning rate scheduling.
5. Batch normalization
6. Plot the accuracy vs epoch, and loss vs epoch for training and testing data.

In [ ]:
# Import TensorFlow and TensorFlow Datasets
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings('ignore') 
import tensorflow as tf
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label


def get_data():
    #Download the dataset and divide it into train and test
    datasets, info = tfds.load('beans', with_info=True, as_supervised=True)
    beans_train,beans_val= datasets['train'],datasets['validation']

    BUFFER_SIZE = 10000

    BATCH_SIZE_PER_REPLICA = 32
    BATCH_SIZE = BATCH_SIZE_PER_REPLICA# * strategy.num_replicas_in_sync
    train_dataset = beans_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    val_dataset = beans_val.map(scale).batch(BATCH_SIZE)
#     test_dataset = beans_test.map(scale).batch(BATCH_SIZE)
    return beans_train,beans_val, train_dataset,val_dataset,info

beans_train,beans_val, train_dataset,val_dataset,info = get_data()

In [ ]:
import matplotlib.pyplot as plt
# 1. View example figures with label in 3 x 3 grids
images = iter(beans_train)
num_classes = info.features['label'].num_classes
print(f"total {num_classes} classes.")
get_label_name = info.features['label'].int2str
i = 0
fig = plt.figure(figsize=(10,9))
for image,label in  iter(beans_train):
    ax = fig.add_subplot(3, 3, i + 1)
    ax.imshow(image)
    plt.title(get_label_name(label))
    ax.axis("off")
    i+=1
    if i>=9:
        break
plt.show()

In [ ]:
def create_model(learning_rate, num_dense_layers, activation,num_classes= 3):
    # Make CNN
    model = tf.keras.Sequential([
                            #     2. Use data augmentation to increase data size
                            tf.keras.Sequential([
                                  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
                                  tf.keras.layers.RandomRotation(0.2),
                                ]),
                            # COV2D layer
                            tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(500, 500, 3)),
                            #     6. Batch normalization
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.MaxPooling2D(),
                            tf.keras.layers.Conv2D(32, 3, activation='relu'),
    #                         tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.MaxPooling2D(),
                            tf.keras.layers.Conv2D(64, 3, activation='relu'),
    #                         tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.MaxPooling2D(),
                            tf.keras.layers.Flatten(),
                            ])
    for i in range(num_dense_layers):
        model.add(tf.keras.layers.Dense(32, activation=activation))
    model.add(tf.keras.layers.Dense(num_classes,activation='softmax'))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
            metrics=['accuracy'])
    return model

In [ ]:
import time
global learning_rate

learning_rate, num_dense_layers, activation = 1e-4,1,'relu'
print(learning_rate, num_dense_layers, activation)
# Define the checkpoint directory to store the checkpoints
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
  
# Callback for printing the LR at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

# 5. Apply early stopping, learning rate scheduling. 
# Function for decaying the learning rate.
def decay(epoch):
    if epoch < 3:
        return learning_rate * 10
    elif epoch >= 3 and epoch < 7:
        return learning_rate
    else:
        return learning_rate / 10

# 4. Use callback to monitor your training.
# Define the callbacks
callbacks = [
#     tf.keras.callbacks.TensorBoard(log_dir='./logs'),
#     tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
#                                        save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    # 5. Apply early stopping, learning rate scheduling. 
    tf.keras.callbacks.EarlyStopping(patience=10, monitor='loss', mode='min', verbose=1, restore_best_weights=False),
    PrintLR()
]

# train the model using the best hyper params
start = time.time()
# create model
model = create_model(learning_rate, num_dense_layers, activation)
# train model
history = model.fit(train_dataset, epochs = 50, 
          validation_data = val_dataset,
          callbacks=callbacks)
end = time.time()
print("Time elapsed: {}".format(end-start))

In [ ]:
# 7. Plot the accuracy vs epoch, and loss vs epoch for training and testing data.
import matplotlib.pyplot as plt
# get training history
loss = history.history['loss']
acc = history.history['accuracy']
val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']
x_epoch = list(range(1,len(loss)+1))
# plot the figure
fig = plt.figure(figsize=(10,6))
ax1 = fig.add_subplot(1,1,1)
ax2 = ax1.twinx()
ax1.plot(x_epoch,acc,'r-*',label = 'train_accuracy')
ax2.plot(x_epoch,loss,'g-*',label = 'train_loss')
ax1.plot(x_epoch,val_acc,'r-o',label = 'val_accuracy')
ax2.plot(x_epoch,val_loss,'g-o',label = 'val_loss')
ax1.legend(loc = 2)
ax2.legend(loc = 0)
ax1.set_ylabel('Accuracy')
ax2.set_ylabel('Loss')
ax1.set_xlabel('Epoches')
plt.title("History")
plt.tight_layout()
plt.show()